to run presentation, enter `jupyter nbconvert Tutorial.ipynb --to slides --post serve` in the terminal

# Text Classification with BERT

<img src="img/bert_reading.jpg" style="width: 500px;"/>

In [3]:
import os
import re

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import numpy as np
np.random.seed(123)

import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm

from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer


BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

## Workshop Goal in Machine Learning Jargon:

_Leverage a **bidirectional language model** with a **transformer architecture** for **text classification** using **transfer learning**_

Right now, this should make no sense!

But by the end of this workshop, you will not only understand what this means, but you will be able to use this method to train a natural language processing model with only 500 data points.

**Note:** This code in this tutorial is largely based on Google's tutorial available [here](https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb) and this Towards Data Science post available [here](https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b).

# The Road Ahead

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for **text classification** using **transfer learning**_

1. Text Classification
2. Transfer Learning
3. Bidirectional Language Model
4. Transformer Architecture
5. Put it all together!

# 1. Text Classification

Text classification is... classifying text.

### Sentiment analysis:
<img src="img/harvard_review.png" style="width: 500px;"/>

### Topic classification
<img src="img/topic_classification.png" style="width: 500px;"/>

## IMDb Sentiment Analysis

Today, we're going to be working with the IMDb sentiment analysis dataset.

**The Task:** Classify a movie review on IMDb as <span style="color:green">positive</span> or <span style="color:red">negative</span>.

## Loading the Data

We will be using the IMDB Large Movie Review Dataset, which is hosted by Stanford.

**TODO: Look into hosting data ourselves**

In [5]:
from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in tqdm(os.listdir(directory), desc=os.path.basename(directory)):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
    train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

    return train_df, test_df

We now define a class that will handle downloading and preparing the text

```python
class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "polarity"

    def __init__(self, tokenizer: FullTokenizer, sample_size=None, max_seq_len=128):
        # Download the data
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        train, test = download_and_load_datasets()
        
        # Re-index by length of text
        train, test = map(lambda df: 
                          df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index), 
                          [train, test])
        
        # sample the data
        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.head(sample_size), test.head(sample_size)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])
        
        # prepare the data
        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [train, test])
        
        # pad the data as needed
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad, 
                                                       [self.train_x, self.test_x])
```

```python
def _prepare(self, df):
    x, y = [], []
    with tqdm(total=df.shape[0], unit_scale=True) as pbar:
        for ndx, row in df.iterrows():
            # get text and label from row of data
            text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]

            # tokenize
            tokens = self.tokenizer.tokenize(text)

            # append special start and end tokens
            tokens = ["[CLS]"] + tokens + ["[SEP]"]

            # convert tokens to IDs
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

            # clip texts to keep it under the max sequence length threshold
            self.max_seq_len = max(self.max_seq_len, len(token_ids))

            # save results
            x.append(token_ids)
            y.append(int(label))
            pbar.update()
    return np.array(x), np.array(y)
```

```python
def _pad(self, ids):
        # The function pads the texts if they are less than the max sequence length threshold
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)
```

In [6]:
class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "polarity"

    def __init__(self, tokenizer, sample_size=None, max_seq_len=128):
        # Download the data
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        train, test = download_and_load_datasets()
        
        # Re-index by length of text
        train, test = map(lambda df: df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index), 
                          [train, test])
        
        # sample the data
        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.sample(sample_size), test.sample(sample_size)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])
        
        # prepare the data
        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [train, test])
        
        # pad the data as needed
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad, 
                                                       [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
        with tqdm(total=df.shape[0], unit_scale=True) as pbar:
            for ndx, row in df.iterrows():
                # get text and label from row of data
                text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]
                
                # tokenize
                tokens = self.tokenizer.tokenize(text)
                
                # append special start and end tokens
                tokens = ["[CLS]"] + tokens + ["[SEP]"]
                
                # convert tokens to IDs
                token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                
                # clip texts to keep it under the max sequence length threshold
                self.max_seq_len = max(self.max_seq_len, len(token_ids))
                
                # save results
                x.append(token_ids)
                y.append(int(label))
                pbar.update()
        return np.array(x), np.array(y)

    def _pad(self, ids):
        # The function pads the texts if they are less than the max sequence length threshold
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)

## We'll come back to this later...

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for <span style="color:green;font-weight: bold">text classification</span> using **transfer learning**_

# 2. Transfer Learning

This was yesterday's topic so we won't go into too much detail.

Transfer learning "focuses on storing knowledge gained while solving one problem and applying it to a different but related problem" (Wikipedia).

<div style="display: flex">
    <img src="img/math.jpeg" style="width: 400px;"/>
    <img src="img/cs.jpeg" style="width: 400px;"/>
</div>

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for <span style="color:green;font-weight: bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

# 3. Bidirectional Language Model

Bidirectional language modelling will be our "first task" in the transfer learning framework. 

We will transfer what we learn from bidirectional language modelling to the IMDb sentiment analysis task.

## But What Is Bidirectional Language Modelling?

## But What Is ~~Bidirectional~~ Language Modelling?


<img src="img/LM.jpeg"/>


## But What Is Bidirectional Language Modelling?

<img src="img/BiLM.png"/>


This task is self-supervised, which means it can be scaled to enormous amounts of data!

**Goal:** _Leverage a <span style="color:green;font-weight: bold">bidirectional language model</span> with a **transformer architecture** for <span style="color:green;font-weight:bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

# 4. Transformer Architecture

Before you can understand transformers, you need to understand attention.

## Dot Product Attention
<img src="img/attention.png"/>
https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3

## Dot Product Attention

Can capture long-range dependencies better than recurrent models.

## Transformers
<img src="img/attention_all.png" style="width:600px"/>

## Transformers
<img src="img/transformer.png" style="width:300px"/>
Improving Language Understanding by Generative Pre-Training

## Self Attention
It's almost attention, but everything is in terms of the input tokens.

<img src="img/attention.png"/>


# BERT

BERT basically just uses a transformer architecture for bidirectional language modelling.

Trained on over 3 billion words!

Google has released the model on tensorflow-hub.

To transfer BERT to sentence level tasks like sentiment analysis, we represent the sentence with the encoding for the class token.

<img src="img/BiLM.png"/>


**Goal:** _Leverage a <span style="color:green;font-weight: bold">bidirectional language model</span> with a <span style="color:green;font-weight: bold">transformer architecture</span> for <span style="color:green;font-weight:bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

Hurray! Now we can code it up.

# 5. Put it all together!

## Load BERT

In [7]:
bert_ckpt_dir="gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/"
bert_ckpt_file = bert_ckpt_dir + "bert_model.ckpt"
bert_config_file = bert_ckpt_dir + "bert_config.json"

In [8]:
%%time

bert_model_dir="2018_10_18"
bert_model_name="uncased_L-12_H-768_A-12"

!mkdir -p .model .model/$bert_model_name

for fname in ["bert_config.json", "vocab.txt", "bert_model.ckpt.meta", "bert_model.ckpt.index", "bert_model.ckpt.data-00000-of-00001"]:
  cmd = f"gsutil cp gs://bert_models/{bert_model_dir}/{bert_model_name}/{fname} .model/{bert_model_name}"
  !$cmd

!ls -la .model .model/$bert_model_name

/bin/sh: gsutil: command not found
/bin/sh: gsutil: command not found
/bin/sh: gsutil: command not found
/bin/sh: gsutil: command not found
/bin/sh: gsutil: command not found
.model:
total 0
drwxr-xr-x   3 joshfeldman  staff   96 14 Jan 10:07 .
drwxr-xr-x  14 joshfeldman  staff  448 14 Jan 10:12 ..
drwxr-xr-x   2 joshfeldman  staff   64 14 Jan 10:07 uncased_L-12_H-768_A-12

.model/uncased_L-12_H-768_A-12:
total 0
drwxr-xr-x  2 joshfeldman  staff  64 14 Jan 10:07 .
drwxr-xr-x  3 joshfeldman  staff  96 14 Jan 10:07 ..
CPU times: user 20.5 ms, sys: 46 ms, total: 66.4 ms
Wall time: 820 ms


In [ ]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="mask_ids")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
                            output_shape=[20],  # Outputs a tensor with shape [batch_size, 20].
                            input_shape=[]
                            trainable=True,
                            signature="tokens")

In [16]:
bert_layer.

In [10]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

TypeError: in converted code:

    /Users/joshfeldman/Harvard/ComputeFest/BERT_tutorial_env/lib/python3.6/site-packages/tensorflow_hub/keras_layer.py:209 call  *
        result = f()
    /Users/joshfeldman/Harvard/ComputeFest/BERT_tutorial_env/lib/python3.6/site-packages/tensorflow_core/python/eager/function.py:1551 __call__  *
        return self._call_impl(args, kwargs)
    /Users/joshfeldman/Harvard/ComputeFest/BERT_tutorial_env/lib/python3.6/site-packages/tensorflow_core/python/eager/function.py:1570 _call_impl
        ).format(self._num_positional_args, self._arg_keywords, args))

    TypeError: Expected at most 0 positional arguments (and the rest keywords, of ['segment_ids', 'input_mask', 'input_ids']), got ([<tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'mask_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'segment_ids_5:0' shape=(None, 128) dtype=int32>],). When calling a concrete function, positional arguments may not be bound to Tensors within nested 

In [ ]:
bert_layer

In [7]:
bert_layer.

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer


First, we download BERT from Tensorflow Hub.

In [22]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
bert_layer = hub.KerasLayer(bert_module)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [23]:
tokenization_info

{'do_lower_case': <tf.Tensor 'module_4_apply_tokenization_info/Const:0' shape=() dtype=bool>,
 'vocab_file': <tf.Tensor 'module_4_apply_tokenization_info/vocab_file:0' shape=() dtype=string>}

## Data Preprocessing

Load data as an InputExample object.

In [9]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


Download the model and the 

In [24]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [26]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 500


INFO:tensorflow:Writing example 0 of 500


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


## Loading BERT

In [15]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
    bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
    bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for sentiment data.
    output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

    
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
            is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
            loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                    label_ids,
                    predicted_labels)
                auc = tf.metrics.auc(
                    label_ids,
                    predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels) 
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)   
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)  
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
            'probabilities': log_probs,
            'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

## Get Ready for Fine Tuning

In [16]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
# SAVE_CHECKPOINTS_STEPS = 500
# SAVE_SUMMARY_STEPS = 100

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# # Specify outpit directory and number of checkpoint steps to save
# run_config = tf.estimator.RunConfig(
#     model_dir=OUTPUT_DIR,
#     save_summary_steps=SAVE_SUMMARY_STEPS,
#     save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [17]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  #config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/bv/tqd8yzws73j5hszbpwd8vkk80000gn/T/tmprws1vntv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13362b588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/bv/tqd8yzws73j5hszbpwd8vkk80000gn/T/tmprws1vntv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13362b588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

## Time to Train!

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

# Validation

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)


# Make Your Own Predictions

In [ ]:
def getPrediction(in_sentences):
    labels = ["Negative", "Positive"]
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

predictions = getPrediction(pred_sentences)
predictions